<a href="https://colab.research.google.com/github/igedeaguss/Magang-Pharos/blob/main/coba_lagi_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rapidfuzz
import pandas as pd
import numpy as np
import re
from rapidfuzz import fuzz, process
import warnings
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.8 MB/s eta 0:00:00


In [2]:
data_ims = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/IMS Compiled Q3 2019 - Q2 2024 (List Product).xlsx', sheet_name='Sheet1')
data_survey = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Hospital Survey Product Name Pivot.xlsx', sheet_name='Sheet1')

data_prdk_survey = data_survey.copy()
data_ims_prdk = data_ims.copy()

In [3]:
#Membersihkan nama brand IMS
def clean_product_name(name):
    return name.split(' -')[0].strip().lower() if isinstance(name, str) else ""

# Daftar bentuk sediaan yang diakui
BENTUK_SEDIAAN = [
    "TAB", "CAP", "SYRUP", "SYR", "INJ", "CREAM", "GEL", "SUSP", "EFF", "ODT",
    "PWD", "OINT", "SUPPOS", "INF", "DRP", "CONC", "SOL", "AMP", "ML", "MG", "IU", "MCG", "G"
]

def clean_product_name_survey(product_name):
    """
    Membersihkan nama produk dengan aturan:
    1. Menghapus produk jika mengandung kata 'racikan' atau 'racik'.
    2. Menghapus teks dalam tanda kurung, kecuali jika mengandung angka atau bentuk sediaan.
    3. Jika tanda kurung berisi angka atau bentuk sediaan, hapus tanda kurungnya tetapi biarkan isinya.
    """
    if not isinstance(product_name, str) or product_name.strip() == '':
        return None  # Abaikan jika input kosong atau bukan string

    product_name = product_name.strip()

    # Jika nama produk mengandung "racikan" atau "racik", maka diabaikan (return None)
    if re.search(r'\bracik(an)?\b', product_name, re.IGNORECASE):
        return None

    # Fungsi untuk memproses tanda kurung
    def process_parentheses(match):
        content = match.group(1).strip()
        contains_number = bool(re.search(r'\d', content))  # Cek apakah ada angka
        contains_form = any(sediaan in content.upper() for sediaan in BENTUK_SEDIAAN)  # Cek apakah ada bentuk sediaan

        if contains_number or contains_form:
            return f" {content} "  # Hilangkan tanda kurung, tetapi biarkan isinya
        return ''  # Hapus jika tidak mengandung angka atau bentuk sediaan

    # Hapus tanda kurung yang tidak mengandung angka/bentuk sediaan atau hilangkan kurungnya jika cocok
    cleaned_name = re.sub(r'\((.*?)\)', process_parentheses, product_name)

    # Hilangkan spasi ganda yang mungkin muncul
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip()

    return cleaned_name


#Standarisasi bentuk sediaan
def extract_dosage_form(product_name):
    dosage_form_map = {
        r'\btabs?\b|\btablets?\b|\btablet\b': 'TAB',
        r'\bcaps?\b|\bcapsules?\b|\bcapsul\b': 'CAP',
        r'\bkapsul\b|\bkapsule\b|\bkap\b': 'CAP',
        r'\bsuspensi\b|\bsuspension\b|\bsusp\b': 'SUSP',
        r'\bserbuk\b|\bbubuk\b|\bpowder\b|\bpwd\b': 'PWD',
        r'\boint\b|\bointment\b|\bsalep\b': 'OINT',
        r'\bkaplet\b|\bcaplets?\b|\bkapl\b|\bcaplet\b': 'CAP',
        r'\binj\b|\binjeksi\b|\binjection\b': 'INJ',
        r'\bamp\b|\bampule\b|\bampul\b': 'AMP',
        r'\bcream\b|\bcrm\b|\bkrim\b|\bcr\b': 'CRM',
        r'\bsupp\b|\bsuppositoria\b|\bsuppos\b': 'SUPPOS',
        r'\binf\b|\binfusion\b|\binfus\b': 'INF',
        r'\bsyrup\b|\bsirup\b|\bsyr\b': 'SYR',
        r'\bdrop\b|\bdrops\b|\btetes\b': 'DRP',
        r'\bconc\b|\bconcentrate\b': 'CONC',
        r'\bsol\b|\bsolution\b': 'SOL',
        r'\bgel\b': 'GEL'
    }

    if not isinstance(product_name, str) or product_name.strip() == '':
        return ''

    # Ubah bentuk sediaan dalam nama produk
    for pattern, replacement in dosage_form_map.items():
        product_name = re.sub(pattern, replacement, product_name, flags=re.IGNORECASE)

    return product_name  # Tidak langsung uppercase agar mudah dibaca dalam tahap sebelumnya

# Standarisasi: Ubah "syringe" menjadi "inj" dalam data IMS
data_ims_prdk["Standardized Name"] = data_ims_prdk["Pack"].str.replace(r"\bsyringe\b", "inj", regex=True)


# Kata kunci alat kesehatan
medical_device_keywords = [
    "stop cock", "catheter", "stent", "scalpel", "forceps", "retractor",
    "bandage", "suture", "gloves", "gown", "dental", "lens", "surgical", "probe", "monitor",
    "electrode", "defibrillator", "ventilator", "prosthesis", "wire", "needle", "screw", "plate",
    "straw", "catether", "selang", "stopcock", "incisor", "sponge", "locking", "accu check", "accucheck", "ACCU-CHEK"
    "arm sling", "action reliever", "active lite", "adult bacterial", "oxygenator", "adult rem",
    "advantime", "aesculap", "affinity", "alas", "alcohol swab", "allevyn adhesive","face mask", "GYNECOLOGY SET","ALERT SNAP"
]

# Fungsi untuk mengecek apakah produk alat kesehatan
def is_medical_device(product_name):
    """Cek apakah nama produk merupakan alat kesehatan"""
    product_name_lower = str(product_name).lower()

    # Jika produk ada di IMS, maka bukan alat kesehatan
    if product_name in data_ims_prdk["Pack"].values:
        return False

    # Jika mengandung kata kunci alat kesehatan, maka termasuk alat kesehatan
    return any(keyword in product_name_lower for keyword in medical_device_keywords)

#Standarisasi Nama Produk di Dataset Utama
data_ims_prdk["Standardized Name"] = data_ims_prdk["Pack"].apply(lambda x:
    data_ims_prdk.loc[data_ims_prdk["Pack"] == x, "Standardized Name"].values[0]
    if x in data_ims_prdk["Pack"].values else x
)

#Buat Kolom "Cleaned Product" (hapus jika alat kesehatan)
data_ims_prdk["Cleaned Product"] = data_ims_prdk["Standardized Name"].apply(lambda x: x if not is_medical_device(x) else np.nan)

# Terapkan ke dataset
data_ims_prdk['Cleaned Brand Name'] = data_ims_prdk['Product'].apply(clean_product_name)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Product Name'].apply(clean_product_name_survey)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].apply(lambda x: x if not is_medical_device(x) else np.nan)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].apply(extract_dosage_form)


#Pastikan semua hasil akhir dalam huruf besar
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].str.upper()

In [6]:
data_prdk_survey[data_prdk_survey['Product Name'].str.contains("incisor", case=False)]

,Kode,Product Name,Cleaned Name
74,75,4.5MM INCISOR PLUS ELITE,


In [14]:
survey_1000 = data_prdk_survey[:100]

In [ ]:
# 1️⃣ Fungsi untuk standarisasi nama produk
def standardize_product_name(name):
    if pd.isna(name):
        return ""

    # Hapus tanda kurung kecuali yang mengandung dosis atau satuan
    pattern = r"\((?!\d+\s?(ML|MG|IU|G|MCG|L|KG|M[LI]U|CAP|TAB|VIAL|AMP|PFS|SUP|DRP|TUBE|PATCH|SACH|BAG|BOTTLE))[^)]*\)"
    name = re.sub(pattern, "", name)

    # Hapus karakter khusus kecuali angka, huruf, '%', dan '/'
    name = re.sub(r"[^a-zA-Z0-9\s%/]", "", name)

    return name.strip()

# 2️⃣ Fungsi ekstraksi angka + satuan
def extract_numbers_and_units(name):
    matches = re.findall(r'(\d+\.?\d*)\s?([A-Za-z%/]*)', name)
    return {f"{num} {unit}".strip() for num, unit in matches}

# 3️⃣ Fungsi pencocokan angka + satuan
def match_numbers_with_units(survey_numbers, pack_numbers):
    if not survey_numbers or not pack_numbers:
        return 0

    intersection = survey_numbers.intersection(pack_numbers)
    return (len(intersection) / max(len(survey_numbers), len(pack_numbers))) * 100

# 4️⃣ Fungsi pencocokan bentuk sediaan
def match_dosage_form(survey_str, ims_str):
    common_forms = ["SYRUP", "SYR DRY", "TAB", "CAP", "INJ", "POWDER", "PWD", "SYR", "CRM", "SUSP",
                    "AMP", "OINT", "DRP", "CONC", "SOL", "GEL"]

    survey_form = next((form for form in common_forms if form in survey_str.upper()), "")
    ims_form = next((form for form in common_forms if form in ims_str.upper()), "")

    return fuzz.partial_ratio(survey_form, ims_form) if survey_form and ims_form else 50

# 5️⃣ Fungsi pencocokan berdasarkan setiap kata dalam brand
def match_brand_words(brand_words, product_name):
    product_brand = product_name.split()[0].lower()  # Ambil kata pertama dari produk (kemungkinan brand utama)

    best_match_score = 0
    for brand in brand_words:
        brand = brand.lower()
        score = fuzz.partial_ratio(brand, product_brand)  # Cek kesamaan brand secara fuzzy
        best_match_score = max(best_match_score, score)  # Simpan skor tertinggi

    return best_match_score

# 6️⃣ Fungsi utama untuk menghitung kemiripan
def calculate_similarity(cleaned_survey_name, pack_list, brand_words):
    results = []
    survey_numbers = extract_numbers_and_units(cleaned_survey_name)

    for pack_tuple in pack_list:
        pack_name, molecule, brand = pack_tuple
        pack_numbers = extract_numbers_and_units(pack_name)

        # Skor angka + satuan
        score_numbers = match_numbers_with_units(survey_numbers, pack_numbers)
        # Skor string berbasis fuzzy matching
        score_strings = fuzz.partial_ratio(cleaned_survey_name, pack_name)
        # Skor bentuk sediaan
        score_dosage_form = match_dosage_form(cleaned_survey_name, pack_name)
        # Skor pencocokan brand (baru)
        score_brand = match_brand_words(brand_words, pack_name)

        # Gabungkan skor dengan bobot baru
        combined_score = (0.3 * score_numbers) + (0.4 * score_strings) + (0.2 * score_dosage_form) + (0.1 * score_brand)

        results.append((pack_name, molecule, brand, combined_score, len(pack_name)))

    # Urutkan berdasarkan skor kemiripan dan panjang nama
    results.sort(key=lambda x: (x[3], x[4]), reverse=True)
    return results

# 7️⃣ Fungsi utama untuk pencocokan produk berdasarkan brand IMS
def process_brand_matching(data_ims, data_survey):
    results = []

    unique_brands = data_ims['Cleaned Brand Name'].dropna().unique()

    for brand_keyword in unique_brands:
        if re.search(r'\bracikan\b|\bracik\b', brand_keyword, re.IGNORECASE):
            continue

        # 💡 **Hanya gunakan kata yang bukan angka & lebih dari 3 karakter**
        brand_words = [word for word in brand_keyword.lower().split() if len(word) > 3 and not word.isnumeric()]

        # Lewati pencarian jika tidak ada kata tersisa
        if not brand_words:
            continue

        regex_pattern = r"(?<![\w\d])(" + r"|".join(map(re.escape, brand_words)) + r")(?![\w\d])"

        # Filter hanya produk survey yang mengandung setidaknya satu kata dari brand
        filtered_survey = data_survey[
            data_survey['Cleaned Name'].str.lower().str.contains(regex_pattern, na=False, regex=True)
        ]

        if filtered_survey.empty:
            continue

        # Ambil hanya produk IMS yang sesuai dengan brand ini
        filtered_ims = data_ims[data_ims['Cleaned Brand Name'] == brand_keyword]
        pack_list = list(zip(filtered_ims['Pack'], filtered_ims['Pack Molecule String'], filtered_ims['Cleaned Brand Name']))

        for _, survey_row in filtered_survey.iterrows():
            cleaned_survey_name = survey_row['Cleaned Name']
            produk_survey = survey_row['Product Name']

            # Hitung kemiripan
            similarity_scores = calculate_similarity(cleaned_survey_name, pack_list, brand_words)

            # Ambil hasil terbaik
            if similarity_scores:
                best_match = similarity_scores[0]
                best_match_pack, best_molecule, best_brand, best_score, _ = best_match
                results.append({
                    'Product Survey': produk_survey,
                    'Best Match IMS Pack': best_match_pack,
                    'Pack Molecule String': best_molecule,
                    'Brand': best_brand,
                    'Best Similarity Score': best_score
                })
            else:
                print(f"Tidak ada pencocokan untuk produk: {produk_survey}")

    return pd.DataFrame(results)



# 8️⃣ Eksekusi Program
filtered_results = process_brand_matching(data_ims_prdk, data_prdk_survey)

# Ambil hanya hasil dengan skor tertinggi untuk setiap produk survey
if not filtered_results.empty:
    filtered_results = filtered_results.loc[filtered_results.groupby('Product Survey')['Best Similarity Score'].idxmax()]

filtered_results

,Product Survey,Best Match IMS Pack,Pack Molecule String,Brand,Best Similarity Score
9684,0.9% NACL 1 LTR ( B BRAUN),ECOSOL NACL INFUSION PLA 0.9% 100ML,SODIUM,ecosol nacl,57.400000
9685,0.9% NACL 1 LTR (OT),ECOSOL NACL INFUSION PLA 0.9% 100ML,SODIUM,ecosol nacl,57.400000
9686,0.9% NACL 1 LTR (WB),ECOSOL NACL INFUSION PLA 0.9% 100ML,SODIUM,ecosol nacl,57.400000
135544,0.9% NACL 10 ML OTSU,OTSU NS AMP 0.9% 10ML,SODIUM,otsu ns,74.827586
9688,0.9% NACL 100 ML (BBRAUN),ECOSOL NACL INFUSION PLA 0.9% 100ML,SODIUM,ecosol nacl,80.769231
...,...,...,...,...,...
131346,zipren 1 mg,ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,51.111111
131348,"zipren 1,5mg",ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,49.473684
131357,"zipren 2,5 mg",ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,48.000000
131359,zipren 2.5 mg,ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,48.000000


In [8]:
filtered_results[filtered_results['Product Survey'].str.contains('aclonac',case=False)]

NameError: name 'filtered_results' is not defined

In [10]:
fuzz.partial_ratio("ACETYLCYSTEIN","ACETYLCYSTEINE")

100.0

In [ ]:
import pandas as pd
import numpy as np
import re

# Kata kunci alat kesehatan
medical_device_keywords = [
    "stop cock", "catheter", "stent", "scalpel", "forceps", "retractor",
    "bandage", "suture", "gloves", "gown", "dental", "lens", "surgical", "probe", "monitor",
    "electrode", "defibrillator","ventilator", "prosthesis","wire","needle",'screw','plate',"straw","catether","selang",
    "stopcock"
]

# Kata kunci obat
medicine_keywords = [
    "tablet", "tab", "capsule", "cap", "syr", "syrup", "inj", "injection", "vial", "cream",
    "ointment", "gel", "drop", "drp", "solution", "suspension", "powder", "mg", "iu"
]

def is_medical_device(product_name):
    """Cek apakah nama produk adalah alat kesehatan"""
    product_name_lower = str(product_name).lower()

    # Pisahkan nama produk menjadi kata-kata
    words = set(re.findall(r'\b\w+\b', product_name_lower))

    # 1️⃣ Cek apakah ada kata kunci alat kesehatan
    contains_medical_keyword = any(keyword in words for keyword in medical_device_keywords)

    # 2️⃣ Cek apakah juga mengandung kata kunci obat
    contains_medicine_keyword = any(keyword in words for keyword in medicine_keywords)

    # 3️⃣ Jika hanya alat kesehatan tanpa kata kunci obat, maka hapus
    return contains_medical_keyword and not contains_medicine_keyword

# Contoh DataFrame
df = pd.DataFrame({
    "Product Name": [
        "3 WAY STOP COCK",
        "STOP COLD SYR 60ML",
        "INFLUVAC SYRINGE 0.5ML",
        "LANOSAN PLUS EYE DRP 2.5ML",
        "SURGICAL GOWN",
        "BLOOD PRESSURE MONITOR",
        "OB IBU& ANAK SYR 75ML",
        "B.DISP SYRINGE 1CC TBCLN27G",
        "3.5 LOCKING SCREW 40MM TITANIUM (218.0103540.28)",
        "ABILIFY MAINTENA 400 MG SUSP INJ"
    ]
})

# Buat kolom "Cleaned Product" hanya untuk produk yang bukan alat kesehatan
df["Cleaned Product"] = df["Product Name"].apply(lambda x: x if not is_medical_device(x) else np.nan)

# Hasil Output
df

,Product Name,Cleaned Product
0,3 WAY STOP COCK,3 WAY STOP COCK
1,STOP COLD SYR 60ML,STOP COLD SYR 60ML
2,INFLUVAC SYRINGE 0.5ML,INFLUVAC SYRINGE 0.5ML
3,LANOSAN PLUS EYE DRP 2.5ML,LANOSAN PLUS EYE DRP 2.5ML
4,SURGICAL GOWN,NaN
5,BLOOD PRESSURE MONITOR,NaN
6,OB IBU& ANAK SYR 75ML,OB IBU& ANAK SYR 75ML
7,B.DISP SYRINGE 1CC TBCLN27G,B.DISP SYRINGE 1CC TBCLN27G
8,3.5 LOCKING SCREW 40MM TITANIUM (218.0103540.28),NaN
9,ABILIFY MAINTENA 400 MG SUSP INJ,ABILIFY MAINTENA 400 MG SUSP INJ


In [ ]:
standardize_product_name("ADONA AC-17 TAB 30MGFORT 10 10")

'ADONA AC17 TAB 30MGFORT 10 10'

In [ ]:
import os

file_name = f"04032025 Data Match Survey and IMS.xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
filtered_results.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/04032025 Data Match Survey and IMS.xlsx


# Pencocokan dengan ambang batas 85%



In [16]:
# 1️⃣ Fungsi untuk standarisasi nama produk
def standardize_product_name(name):
    if pd.isna(name):
        return ""

    # Hapus tanda kurung kecuali yang mengandung dosis atau satuan
    pattern = r"\((?!\d+\s?(ML|MG|IU|G|MCG|L|KG|M[LI]U|CAP|TAB|VIAL|AMP|PFS|SUP|DRP|TUBE|PATCH|SACH|BAG|BOTTLE))[^)]*\)"
    name = re.sub(pattern, "", name)

    # Hapus karakter khusus kecuali angka, huruf, '%', dan '/'
    name = re.sub(r"[^a-zA-Z0-9\s%/]", "", name)

    return name.strip()

# 2️⃣ Fungsi ekstraksi angka + satuan
def extract_numbers_and_units(name):
    matches = re.findall(r'(\d+\.?\d*)\s?([A-Za-z%/]*)', name)
    return {f"{num} {unit}".strip() for num, unit in matches}

# 3️⃣ Fungsi pencocokan angka + satuan
def match_numbers_with_units(survey_numbers, pack_numbers):
    if not survey_numbers or not pack_numbers:
        return 0

    intersection = survey_numbers.intersection(pack_numbers)
    return (len(intersection) / max(len(survey_numbers), len(pack_numbers))) * 100

# 4️⃣ Fungsi pencocokan bentuk sediaan
def match_dosage_form(survey_str, ims_str):
    common_forms = ["SYRUP", "SYR DRY", "TAB", "CAP", "INJ", "POWDER", "PWD", "SYR", "CRM", "SUSP",
                    "AMP", "OINT", "DRP", "CONC", "SOL", "GEL"]

    survey_form = next((form for form in common_forms if form in survey_str.upper()), "")
    ims_form = next((form for form in common_forms if form in ims_str.upper()), "")

    return fuzz.partial_ratio(survey_form, ims_form) if survey_form and ims_form else 50



# 6️⃣ Fungsi utama untuk menghitung kemiripan
def calculate_similarity(cleaned_survey_name, pack_list):
    results = []
    survey_numbers = extract_numbers_and_units(cleaned_survey_name)  # Pindah ke luar loop

    for pack_tuple in pack_list:
        pack_name, molecule, brand = pack_tuple
        pack_numbers = extract_numbers_and_units(pack_name)

        # Skor angka + satuan
        score_numbers = match_numbers_with_units(survey_numbers, pack_numbers)
        # Skor string berbasis fuzzy matching
        score_strings = fuzz.partial_ratio(cleaned_survey_name, pack_name)
        # Skor bentuk sediaan
        score_dosage_form = 100 if cleaned_survey_name in pack_name else 50  # Lebih sederhana

        # Gabungkan skor tanpa score_brand
        combined_score = (0.4 * score_numbers) + (0.5 * score_strings) + (0.1 * score_dosage_form)

        results.append((pack_name, molecule, brand, combined_score, len(pack_name)))

    # Urutkan berdasarkan skor kemiripan dan panjang nama
    results.sort(key=lambda x: (x[3], x[4]), reverse=True)
    return results


# 7️⃣ Fungsi utama untuk pencocokan produk berdasarkan brand IMS
def process_brand_matching(data_ims, data_survey, threshold=85):
    results = []

    unique_brands = data_ims['Cleaned Brand Name'].dropna().unique()

    for brand_keyword in unique_brands:
        if re.search(r'\bracikan\b|\bracik\b', brand_keyword, re.IGNORECASE):
            continue

        # 💡 Hanya gunakan kata yang bukan angka & lebih dari 3 karakter
        brand_words = [word for word in brand_keyword.lower().split() if len(word) > 3 and not word.isnumeric()]

        # Lewati pencarian jika tidak ada kata tersisa
        if not brand_words:
            continue

        # 💡 **Gunakan RapidFuzz untuk mencocokkan produk survey**
        filtered_survey = filter_survey_by_brand(data_survey, brand_keyword, threshold)

        if filtered_survey.empty:
            continue

        # Ambil hanya produk IMS yang sesuai dengan brand ini
        filtered_ims = data_ims[data_ims['Cleaned Brand Name'] == brand_keyword]
        pack_list = list(zip(filtered_ims['Pack'], filtered_ims['Pack Molecule String'], filtered_ims['Cleaned Brand Name']))

        for _, survey_row in filtered_survey.iterrows():
            cleaned_survey_name = survey_row['Cleaned Name']
            produk_survey = survey_row['Product Name']

            # Hitung kemiripan
            similarity_scores = calculate_similarity(cleaned_survey_name, pack_list)

            # Ambil hasil terbaik
            if similarity_scores:
                best_match = similarity_scores[0]
                best_match_pack, best_molecule, best_brand, best_score, _ = best_match
                results.append({
                    'Product Survey': produk_survey,
                    'Best Match IMS Pack': best_match_pack,
                    'Pack Molecule String': best_molecule,
                    'Brand': best_brand,
                    'Best Similarity Score': best_score
                })

    return pd.DataFrame(results)


# 7️⃣a Fungsi Fuzzy Matching dengan RapidFuzz
def filter_survey_by_brand(data_survey, brand_name, threshold=85):
    brand_name_lower = brand_name.lower()  # Nama brand dari IMS

    def fuzzy_match(survey_name):
        if not isinstance(survey_name, str):
            return False

        # Gunakan RapidFuzz untuk mencari skor kemiripan
        score = fuzz.partial_ratio(brand_name_lower, survey_name.lower())
        return score >= threshold  # Hanya ambil yang skornya di atas threshold

    # **OPTIMASI: Gunakan apply() untuk mengganti looping for**
    return data_survey[data_survey['Cleaned Name'].apply(fuzzy_match)]



# 8️⃣ Eksekusi Program
filtered_results = process_brand_matching(data_ims_prdk, data_prdk_survey)

# Ambil hanya hasil dengan skor tertinggi untuk setiap produk survey
if not filtered_results.empty:
    filtered_results = filtered_results.loc[filtered_results.groupby('Product Survey')['Best Similarity Score'].idxmax()]

filtered_results

,Product Survey,Best Match IMS Pack,Pack Molecule String,Brand,Best Similarity Score
11962,2 KDT RIFAMPICIN/INH 150/75 MG TABLET,RIFAMPICIN CAP 150MG 100,RIFAMPICIN,rifampicin,38.333333
31894,40% DEXTROSE 25 ML^,DEXTROSE INFUSION 5% 500ML,GLUCOSE,dextrose,37.142857
24580,5% DEXTROSE 1/4 NS,DEXTROSE + 1/2 NS INFUSION 5% 500ML 20,GLUCOSE+SODIUM,dextrose + 1/2 ns,61.000000
24581,5% DEXTROSE 1/4 NS,DEXTROSE + 1/2 NS INFUSION 5% 500ML 20,GLUCOSE+SODIUM,dextrose + 1/2 ns,61.000000
24582,5% DEXTROSE 1/4 NS ( SANBE ),DEXTROSE + 1/2 NS INFUSION 5% 500ML 20,GLUCOSE+SODIUM,dextrose + 1/2 ns,61.000000
...,...,...,...,...,...
68956,zipren 1 mg,ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,43.888889
68958,"zipren 1,5mg",ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,41.842105
68967,"zipren 2,5 mg",ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,40.000000
68969,zipren 2.5 mg,ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,40.000000


In [24]:
import os

file_name = f"Data Match Survey and IMS (Ambang Batas 85%).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
filtered_results.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/Data Match Survey and IMS (Ambang Batas 85%).xlsx


In [23]:
filtered_results[filtered_results['Product Survey'].str.contains('amiodaron', case=False)]

,Product Survey,Best Match IMS Pack,Pack Molecule String,Brand,Best Similarity Score
17250,AMIODARON 150 MG INJ,AMIODARONE INFUSION AMP 150MG/3ML 3ML 6,AMIODARONE,amiodarone,41.363636
17251,AMIODARON 200MG TAB,AMIODARONE TAB 200MG 100 STRP,AMIODARONE,amiodarone,64.393939
17252,AMIODARON 300 MG,AMIODARONE INFUSION AMP 150MG/3ML 3ML 6,AMIODARONE,amiodarone,42.037037
17253,AMIODARON HYDROCHLORIDE TAB 200MG,AMIODARONE TAB 200MG 30 BLIS,AMIODARONE,amiodarone,56.707317
17254,AMIODARONE 150 MG/3 ML INJEKSI - DARYA,AMIODARONE INFUSION AMP 150MG/3ML 3ML 6,AMIODARONE,amiodarone,62.549020
17256,AMIODARONE HCL 150 MG INJ 3 ML,AMIODARONE INFUSION AMP 150MG/3ML 3ML 6,AMIODARONE,amiodarone,48.565891
17257,AMIODARONE HCL 150 MG INJ 3 ML [Retur],AMIODARONE INFUSION AMP 50MG/ML 3ML 5,AMIODARONE,amiodarone,49.215686
17258,AMIODARONE HCL 150 MG INJEKSI 3 ML,AMIODARONE INFUSION AMP 150MG/3ML 3ML 6,AMIODARONE,amiodarone,48.565891
17259,AMIODARONE HCL 150mg/3ml INJEKSI,AMIODARONE INFUSION AMP 150MG/3ML 3ML 6,AMIODARONE,amiodarone,63.373984
17260,AMIODARONE HCL 50 MG/ML INJ (+),AMIODARONE INFUSION AMP 50MG/ML 3ML 5,AMIODARONE,amiodarone,50.833333
